# 1.1 - SENTINEL Quick Start

**Purpose**: Verify installation and test LLM capabilities

**Author**: SENTINEL Developer  
**Date**: 2025-12-29

---

## Objectives
1. Import and test all major libraries
2. Test Ollama connection
3. Test LangChain RAG pipeline
4. Log experiment to MLflow

## 1. Setup & Imports

In [1]:
# Standard libraries
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Data science stack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ML & MLOps
import mlflow
from sklearn.model_selection import train_test_split

# LLM & RAG
from langchain.llms import Ollama
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

print("✅ All imports successful!")

ImportError: cannot import name 'service' from 'google.protobuf' (C:\Users\LENOVO\Documents\SENTINEL PROJECT\venv\Lib\site-packages\google\protobuf\__init__.py)

## 2. Configuration

In [ ]:
# Project configuration
CONFIG = {
    "project_root": Path.cwd().parent.parent,
    "data_dir": Path("../../data"),
    "models_dir": Path("../../models"),
    
    # MLflow
    "mlflow_uri": "http://localhost:5000",
    "experiment_name": "sentinel-quickstart",
    
    # Ollama
    "ollama_model": "llama3.1:8b-instruct-q4_K_M",
    "ollama_base_url": "http://localhost:11434",
    
    # Embeddings
    "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
    
    # Random seed
    "random_state": 42
}

# Set random seeds
np.random.seed(CONFIG["random_state"])

print("✅ Configuration loaded")
print(f"Project root: {CONFIG['project_root']}")

## 3. Test Ollama Connection

In [ ]:
# Initialize Ollama
llm = Ollama(
    model=CONFIG["ollama_model"],
    base_url=CONFIG["ollama_base_url"],
    temperature=0.1
)

# Test query
test_prompt = "Explain insider trading in 2 sentences."

print("🤖 Testing Ollama...")
print(f"Prompt: {test_prompt}")
print("\nResponse:")

response = llm(test_prompt)
print(response)

print("\n✅ Ollama is working!")

## 4. Test Embedding Model

In [ ]:
# Initialize embeddings
embeddings = HuggingFaceEmbeddings(
    model_name=CONFIG["embedding_model"]
)

# Test embedding
test_text = "POJK regulation about insider trading"
embedding_vector = embeddings.embed_query(test_text)

print(f"Text: {test_text}")
print(f"Embedding dimension: {len(embedding_vector)}")
print(f"First 5 values: {embedding_vector[:5]}")
print("\n✅ Embeddings working!")

## 5. Simple RAG Demo

In [ ]:
# Sample documents (simulating regulatory text)
sample_docs = [
    "Insider trading adalah praktik jual beli saham berdasarkan informasi material yang belum dipublikasikan.",
    "Orang dalam perusahaan dilarang melakukan transaksi saham dalam periode quiet period, yaitu 30 hari sebelum publikasi laporan keuangan.",
    "POJK 30/2016 mengatur tentang transaksi material dan benturan kepentingan.",
    "Bursa Efek Indonesia (BEI) mewajibkan pelaporan transaksi oleh orang dalam perusahaan.",
]

# Create text chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)

chunks = text_splitter.create_documents(sample_docs)

print(f"Created {len(chunks)} chunks")
print("\n✅ Text splitting successful!")

In [ ]:
# Create vector store
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="quickstart_demo"
)

print("✅ Vector store created!")

# Test similarity search
query = "Apa itu quiet period?"
docs = vectorstore.similarity_search(query, k=2)

print(f"\nQuery: {query}")
print("\nTop 2 relevant documents:")
for i, doc in enumerate(docs, 1):
    print(f"{i}. {doc.page_content}")

## 6. RAG Query with LLM

In [ ]:
from langchain.chains import RetrievalQA

# Create RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True
)

# Query
question = "Berapa lama quiet period sebelum publikasi laporan keuangan?"

print(f"Question: {question}")
print("\nProcessing...")

result = qa_chain({"query": question})

print("\n" + "="*50)
print("ANSWER:")
print("="*50)
print(result["result"])

print("\n" + "="*50)
print("SOURCE DOCUMENTS:")
print("="*50)
for i, doc in enumerate(result["source_documents"], 1):
    print(f"{i}. {doc.page_content}")

print("\n✅ RAG pipeline working!")

## 7. Log Experiment to MLflow

In [ ]:
# Set MLflow tracking URI
mlflow.set_tracking_uri(CONFIG["mlflow_uri"])
mlflow.set_experiment(CONFIG["experiment_name"])

with mlflow.start_run(run_name="quickstart-test"):
    # Log parameters
    mlflow.log_param("ollama_model", CONFIG["ollama_model"])
    mlflow.log_param("embedding_model", CONFIG["embedding_model"])
    mlflow.log_param("chunk_size", 200)
    mlflow.log_param("num_documents", len(sample_docs))
    
    # Log metrics
    mlflow.log_metric("embedding_dim", len(embedding_vector))
    mlflow.log_metric("num_chunks", len(chunks))
    
    # Log tags
    mlflow.set_tag("type", "quickstart")
    mlflow.set_tag("status", "success")
    
    print("✅ Experiment logged to MLflow!")
    print(f"\nView at: {CONFIG['mlflow_uri']}")

## 8. Summary

### ✅ What We Tested:
1. **Imports** - All major packages working
2. **Ollama** - Local LLM inference working
3. **Embeddings** - Sentence transformers working
4. **RAG Pipeline** - Retrieval + generation working
5. **MLflow** - Experiment tracking working

### 🎯 Next Steps:
1. **Data Acquisition** - Collect real POJK PDFs
2. **Enhanced RAG** - Build production pipeline
3. **Model Training** - Train anomaly detection
4. **API Development** - Build FastAPI endpoints

### 📚 Resources:
- **Roadmap**: `ROADMAP_QUANT_ENHANCED.md`
- **Structure Guide**: `PROJECT_STRUCTURE_GUIDE.md`
- **Quick Start**: `QUICKSTART.md`

---

**Status**: ✅ Installation verified - Ready for development!